In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
TESTING = False

# Motivation

The motivation for this approach is to use a pretrained image classifier on image representations of the data provided. The reason I'm trying this is that Jeremy Howard of Fast AI provided several examples in his book about deep learning where such approach provided state of the art results. So let's try how it will perform with this problem.

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")

In [ ]:
train.shape, test.shape

In [ ]:
print("TRAIN:")
display(train.head(10))

print("TEST:")
display(test.head(10))

# Convert one row into a 2D array


In [ ]:
import matplotlib.pyplot as plt

feats = train.columns[1:-1]

# Extract row array
sample = train[feats].iloc[0].values

# Min Max Scaling
sample = (sample - train[feats].min(axis=0)) / (train[feats].max(axis=0) - train[feats].min(axis=0))

# Rescale to 255
sample *= 255

# Append 3 extra features to make dim 289 -> (17, 17)
sample = np.append(sample, [0, 0, 0])
sample = np.reshape(sample, (17, 17))

plt.imshow(sample);

Now if we are going to use a ResNet model we'll need to upsize such image into a `224 x 224` array.

In [ ]:
from torchvision.transforms import Resize
from PIL import Image

In [ ]:
transform = Resize((224, 224))

PIL_image = Image.fromarray(np.uint8(sample))
PIL_image = transform(PIL_image)
plt.imshow(PIL_image);

# Convert Training Data into 3D array 

In [ ]:
def convert_df_to_3d(data, train_min_max=None):
    # Add 3 features to enable resizing into 17x17
    data.loc[:, ['extra_0', 'extra_1', 'extra_2']] = [0, 0, 0]
    
    # Min Max Scaling
    if not train_min_max:
        data_min, data_max = data.min(axis=0), data.max(axis=0)
    else:
        data_min, data_max = train_min_max 
        
    data = (data - data_min) / (data_max - data_min)
    
    # Scale to 255
    data *= 255

    # Reshape data into 17x17
    data = data.values.reshape((-1, 17, 17))[..., None]
    data = np.repeat(data, 3, axis=-1)
    
    return data, (data_min, data_max)

In [ ]:
train_data = train[feats]
train_data.shape

# Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_data, train.target.to_list(), test_size=test.shape[0])

if TESTING:
    X_train, y_train = X_train[:64], y_train[:64]
    X_val, y_val = X_val[:64], y_val[:64]

    
X_train, train_min_max = convert_df_to_3d(X_train)
X_val, _ = convert_df_to_3d(X_val, train_min_max)

In [ ]:
tmp = Resize(224)
plt.imshow(tmp(Image.fromarray(np.uint8(X_train[4])).convert('RGB')))

In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
[1, 2,] == None

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data, labels, transform=None, target_transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = Image.fromarray(np.uint8(self.data[idx])).convert('RGB')
        label = self.labels[idx] if self.labels != None else 0
        
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform and self.labels:
            label = self.target_transform.transform([label])
            
        return image, label

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

label_transformer = LabelEncoder().fit(train['target'])

datasets = {source: CustomImageDataset(X, y, transformer, label_transformer) for source, X, y in (('train', X_train, y_train),
                                                                                                  ('val', X_val, y_val))}

dataloaders = {source: DataLoader(dataset, batch_size=64, shuffle=True) for source, dataset in datasets.items()}

dataset_sizes = {source: len(dataset) for source, dataset in datasets.items()}

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(dataloaders['train']))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img[0])
plt.show()
print(f"Label: {label_transformer.inverse_transform(label)[0]}")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, 1)
                    loss = criterion(outputs, labels.squeeze())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(label_transformer.classes_))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

In [ ]:
torch.save(model_ft, 'model_ft')

# Inference

In [ ]:
test_data = test[feats]
test_data, _ = convert_df_to_3d(test_data, train_min_max)
test_data.shape

In [ ]:
test_dataset = CustomImageDataset(test_data, None, transformer, label_transformer)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
model_ft.eval()

test_preds = []
count = 0
for inputs, _ in test_dataloader:
    count += len(inputs)
    inputs, _ = next(iter(test_dataloader))
    inputs = inputs.to(device)
    outputs = model_ft(inputs)
    preds = torch.argmax(outputs, 1)
    test_preds += preds.tolist()
    print(f'{count}/{test.shape[0]}', end='\r', flush=True)

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')
submission['target'] = label_transformer.inverse_transform(test_preds[:test.shape[0]])

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)